# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/chrag/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chrag/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [23]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [25]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [26]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [27]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [28]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [29]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [30]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [31]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/28 [00:00<?, ?it/s]

Property 'summary' already exists in node '79c56b'. Skipping!
Property 'summary' already exists in node 'a8c4ff'. Skipping!
Property 'summary' already exists in node 'c54047'. Skipping!
Property 'summary' already exists in node '3fb1a7'. Skipping!
Property 'summary' already exists in node '433237'. Skipping!
Property 'summary' already exists in node 'e25b10'. Skipping!
Property 'summary' already exists in node 'cbbe30'. Skipping!
Property 'summary' already exists in node '4a8e29'. Skipping!
Property 'summary' already exists in node '0f2be9'. Skipping!
Property 'summary' already exists in node '94c4c9'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/16 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/54 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '79c56b'. Skipping!
Property 'summary_embedding' already exists in node 'a8c4ff'. Skipping!
Property 'summary_embedding' already exists in node '3fb1a7'. Skipping!
Property 'summary_embedding' already exists in node '4a8e29'. Skipping!
Property 'summary_embedding' already exists in node 'c54047'. Skipping!
Property 'summary_embedding' already exists in node '0f2be9'. Skipping!
Property 'summary_embedding' already exists in node 'cbbe30'. Skipping!
Property 'summary_embedding' already exists in node '94c4c9'. Skipping!
Property 'summary_embedding' already exists in node 'e25b10'. Skipping!
Property 'summary_embedding' already exists in node '433237'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 43, relationships: 415)

We can save and load our knowledge graphs as follows.

In [32]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 43, relationships: 415)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [57]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [34]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.
### Answer ✅ ✅ ✅: Single hop generates simple questions answerable by a single fact or document, say 'what was the weather like in Chicago last saturday?'
Multi hop abstract generates questions that need complex abstract reasoning combining multiple soures of data, like how does interest rate reduction impact small businesses? 
Multi hop specific query generates questions that need answers synthesized/ aggregates from multiple sources in a straightfoward manner, like what does the average interest rate of all students in the class of 2012 of x University?




Finally, we can use our `TestSetGenerator` to generate our testset!

In [35]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of FAFSA® in the federal stud...,[Application and Verification Guide Introducti...,The FAFSA® form is used for completing the app...,single_hop_specifc_query_synthesizer
1,Could you please explain how the FSA Knowledge...,[electronically due to limitations on access t...,The FSA Knowledge Center provides resources su...,single_hop_specifc_query_synthesizer
2,What is the Department's role in FAFSA applica...,[Note: There will be two active portals that c...,The Department manages two active portals corr...,single_hop_specifc_query_synthesizer
3,How does the FPS assist students in ensuring t...,[The FPS also checks the application for possi...,The FPS checks the application for possible in...,single_hop_specifc_query_synthesizer
4,How does ED handle output documents like the I...,[Output Documents After processing is complete...,Output documents such as the ISIR and FAFSA Su...,single_hop_specifc_query_synthesizer
5,How does the verification and corrections proc...,[<1-hop>\n\nApplication and Verification Guide...,The verification and corrections of FAFSA info...,multi_hop_abstract_query_synthesizer
6,family and marital status check and veteran st...,[<1-hop>\n\nVeteran of the U.S. Armed Forces T...,The context explains that the FPS checks for f...,multi_hop_abstract_query_synthesizer
7,"How do the recent changes, including the elimi...",[<1-hop>\n\nThe FPS also checks the applicatio...,The elimination of the IRS Data Retrieval Tool...,multi_hop_abstract_query_synthesizer
8,"So, if a student lives with a disabled sibling...","[<1-hop>\n\nLegal Dependent Examples AVG, Chap...","Based on the information from Chapter 2, if a ...",multi_hop_specific_query_synthesizer
9,How does the FSA's implementation of the FAFSA...,[<1-hop>\n\nApplication and Verification Guide...,The FSA's implementation of the FAFSA Simplifi...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [36]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/28 [00:00<?, ?it/s]

Property 'summary' already exists in node '7fa9f4'. Skipping!
Property 'summary' already exists in node '943157'. Skipping!
Property 'summary' already exists in node '683f88'. Skipping!
Property 'summary' already exists in node '2c4db4'. Skipping!
Property 'summary' already exists in node '845c56'. Skipping!
Property 'summary' already exists in node 'd1bcb3'. Skipping!
Property 'summary' already exists in node '5b38f0'. Skipping!
Property 'summary' already exists in node '8a6a49'. Skipping!
Property 'summary' already exists in node 'd609a9'. Skipping!
Property 'summary' already exists in node '4cdb3a'. Skipping!


...

Property 'summary_embedding' already exists in node '683f88'. Skipping!
Property 'summary_embedding' already exists in node '7fa9f4'. Skipping!
Property 'summary_embedding' already exists in node '943157'. Skipping!
Property 'summary_embedding' already exists in node '2c4db4'. Skipping!
Property 'summary_embedding' already exists in node 'd1bcb3'. Skipping!
Property 'summary_embedding' already exists in node '4cdb3a'. Skipping!
Property 'summary_embedding' already exists in node '8a6a49'. Skipping!
Property 'summary_embedding' already exists in node '845c56'. Skipping!
Property 'summary_embedding' already exists in node 'd609a9'. Skipping!
Property 'summary_embedding' already exists in node '5b38f0'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [37]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the purpose of FAFSA in the financial ...,[Application and Verification Guide Introducti...,The FAFSA is used to help students apply for f...,single_hop_specifc_query_synthesizer
1,What recent updates have been made to the FAFS...,[Chapter 1: The Application Process We removed...,The chapter states that the section for Return...,single_hop_specifc_query_synthesizer
2,What role does the FAFSA play in the financial...,[Output Documents After processing is complete...,The FAFSA is used to produce output documents ...,single_hop_specifc_query_synthesizer
3,How does the Department handle the disclosur o...,[2. The disclosure of their FTI by the IRS to ...,The disclosure of their FTI by the IRS to the ...,single_hop_specifc_query_synthesizer
4,How does the FAFSA form determine veteran stat...,[<1-hop>\n\nVeteran of the U.S. Armed Forces T...,The FAFSA form asks students to declare their ...,multi_hop_abstract_query_synthesizer
5,What are the dischare conditions that make a p...,[<1-hop>\n\nVeteran of the U.S. Armed Forces T...,"According to the context, a person is consider...",multi_hop_abstract_query_synthesizer
6,Criteria for homelessness detemination and doc...,[<1-hop>\n\nSubmission of a court order or off...,The criteria for homelessness determination in...,multi_hop_abstract_query_synthesizer
7,How do family information and aid history show...,[<1-hop>\n\nThe FPS also checks the applicatio...,"The output documents, such as the ISIR and FAF...",multi_hop_abstract_query_synthesizer
8,How does the FAFSA Partner Portal facilitate t...,[<1-hop>\n\nOutput Documents After processing ...,The FAFSA Partner Portal allows schools to add...,multi_hop_specific_query_synthesizer
9,Volume 6 and Volume 7 info how they relate for...,[<1-hop>\n\nOutput Documents After processing ...,The context explains that Volume 6 of the FAFS...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [58]:
from langsmith import Client
from uuid import uuid4

# Step 1: Initialize LangSmith client
client = Client()

# Step 2: Define or retrieve dataset
dataset_name = "Loan Synthetic Data"

try:
    langsmith_dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="Loan Synthetic Data for RAG evaluation"
    )
except Exception as e:
    if "409" in str(e):  # Dataset already exists
        langsmith_dataset = client.read_dataset(dataset_name=dataset_name)
    else:
        raise e


We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [44]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [45]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [47]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [48]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [59]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [60]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [51]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [52]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [53]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct PLUS Loan (or student Federal PLUS Loan)\n- Direct Subsidized Loan\n- Direct Unsubsidized Loan\n- Federal Stafford Loans (Subsidized and Unsubsidized)\n- Federal SLS Loans\n- Federal PLUS Loans\n\nNote: Federal Family Education Loan (FFEL) Program loans (Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, Federal PLUS Loans) were made before new FFEL loans ended effective July 1, 2010. New loans under the FFEL program are no longer made, but existing loans under it still count toward limits. Direct Subsidized, Unsubsidized, and PLUS Loans are currently active loan types.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [54]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [55]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm}) #uses build in LangSmith QA rubric

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {  #custom field mapping
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: ### Answer ✅ ✅ ✅: It uses built-in langsmith QA rubric to assess correctness
- `labeled_helpfulness_evaluator`:### Answer ✅ ✅ ✅: Helpfulness evaluator is making sure that the answer (albiet correct), is actually helping user's intented JTBD
- `empathy_evaluator`:### Answer ✅ ✅ ✅: This is focused on the user's sentiments and feelings, and ensuring the response makes the user feel heard

## LangSmith Evaluation

In [61]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'flowery-lip-35' at:
https://smith.langchain.com/o/88393fec-61d3-4f70-8fcc-c531ecb595eb/datasets/5f0836b9-1dfc-4912-afcd-52a5f8db8d49/compare?selectedSessions=ca549d58-25da-4411-b01b-80bc5f7d9b27




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the FSAIC process relate to the FSA i...,"Based on the provided context, the FSAIC (Stud...",None,The FSAIC (Federal Student Aid Information Cen...,0,0,0,3.175283,daea9ce6-8aec-430a-b019-d36b65c87562,6148c0b3-a626-4c71-9a2b-9f929381f1ba
1,how SSA and social security act relate to stud...,The Social Security Administration (SSA) is in...,None,The context explains that the Department match...,1,1,0,3.634283,fa9eb164-ba45-4def-80ac-3eeb4bf57fb0,65b21515-b50a-4889-ab35-669a7ab35ccb
2,Volume 6 and Volume 7 info how they relate for...,"Based on the provided context, Volume 7 of the...",None,The context explains that Volume 6 of the FAFS...,0,0,0,5.919486,58a45906-e55a-4865-bada-a2af496df2c8,964c1f57-89c9-45d7-a112-f5edd91595e9
3,How does the FAFSA Partner Portal facilitate t...,The FAFSA Partner Portal facilitates resolving...,None,The FAFSA Partner Portal allows schools to add...,1,1,0,5.179635,277d21d9-0c29-41a2-b8f9-e94c796894d7,25b6196b-86bd-488f-a187-0d3a8d76be1f
4,How do family information and aid history show...,Family information and aid history shown in th...,None,"The output documents, such as the ISIR and FAF...",1,1,0,6.045431,d5e38725-610f-4f2c-8b91-6cc931a1e782,2da97f0c-9369-4eb6-b803-55a166c9fc2c
5,Criteria for homelessness detemination and doc...,"Based on the provided context, the criteria an...",None,The criteria for homelessness determination in...,1,1,0,10.415352,1f13a8f1-fc6e-4838-9834-d3db4a8fcd54,503c88d5-fa4f-43b2-a242-8d575e5e2b5c
6,What are the dischare conditions that make a p...,"Based on the provided context, a person is gen...",None,"According to the context, a person is consider...",1,0,0,3.380384,30cb3f38-1d61-40ca-b8a3-46d7acf9d16c,fbbb2709-d5fb-49c0-9971-56136e082aeb
7,How does the FAFSA form determine veteran stat...,"Based on the provided context, the FAFSA form ...",None,The FAFSA form asks students to declare their ...,1,1,0,6.455964,330c2806-e931-424a-8f7b-457c1134c2c5,fcdcf19c-fed1-4303-a6c9-5fe7618c31a0
8,How does the Department handle the disclosur o...,"Based on the provided context, the Department ...",None,The disclosure of their FTI by the IRS to the ...,1,0,0,2.627577,4ac5b6f7-372d-44c7-a132-351d494d0929,013c85ad-e709-4fdd-a79f-2e0cb9d42df3
9,What role does the FAFSA play in the financial...,The FAFSA form is the only application student...,None,The FAFSA is used to produce output documents ...,1,1,0,2.150669,fb5d3d75-4022-4a25-a311-d99262f8ac2c,4000bbfa-c388-4488-80ce-67cbf8a0647a


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [62]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [62]:
rag_documents = docs

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?
Answer ✅ ✅ ✅: Larger chunks are slower to process and the similarity score become more averaged, smaller chunk may miss out on simantic similarity, by becoming too narrow. Overlapping preserves context acorss cunks and imprves retrival 

In [64]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?
Answer ✅ ✅ ✅: different models have different dimentions of semantic relationship, higher and more expensive models would have more dimentions to map the semantic relationship against

In [66]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [67]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [68]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [69]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the information provided, the kinds of loans available include:\n\n- Direct PLUS Loans, which are available to graduate and professional students, as well as to the parents of dependent undergraduate students.\n- Direct Subsidized Loans\n- Direct Unsubsidized Loans\n- Student Federal PLUS Loans\n- Federal Stafford Loans (both Subsidized and Unsubsidized)\n- Federal SLS Loans\n- Federal PLUS Loans under the Federal Family Education Loan (FFEL) Program (though no new loans under FFEL are made after July 1, 2010)\n\nI hope this helps clarify the types of loans that are available to you or your loved ones. If you have more questions or need further assistance, please feel free to ask—I'm here to support you."

Finally, we can evaluate the new chain on the same test set!

In [70]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'dear-wood-73' at:
https://smith.langchain.com/o/88393fec-61d3-4f70-8fcc-c531ecb595eb/datasets/5f0836b9-1dfc-4912-afcd-52a5f8db8d49/compare?selectedSessions=40b2f55c-7d4e-49eb-8f65-31dfa5d7dad3




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the FSAIC process relate to the FSA i...,Thank you for your thoughtful question. Based ...,None,The FSAIC (Federal Student Aid Information Cen...,0,0,1,2.891102,daea9ce6-8aec-430a-b019-d36b65c87562,169f4bdf-6a01-49a5-a1ad-9aea579a2ec3
1,how SSA and social security act relate to stud...,Thank you for your thoughtful question. Based ...,None,The context explains that the Department match...,1,1,1,4.422985,fa9eb164-ba45-4def-80ac-3eeb4bf57fb0,2b229fa3-4dbb-4f1d-9faa-32a9f553656c
2,Volume 6 and Volume 7 info how they relate for...,Thank you for your thoughtful question. Based ...,None,The context explains that Volume 6 of the FAFS...,0,0,1,5.781674,58a45906-e55a-4865-bada-a2af496df2c8,949873ac-a5bc-4922-82b5-669912168f5b
3,How does the FAFSA Partner Portal facilitate t...,Thank you for your thoughtful question. Based ...,None,The FAFSA Partner Portal allows schools to add...,1,1,1,4.842501,277d21d9-0c29-41a2-b8f9-e94c796894d7,71c83027-2636-43de-806c-b4b66b83ec06
4,How do family information and aid history show...,Thank you for your thoughtful question—it's cl...,None,"The output documents, such as the ISIR and FAF...",1,1,1,3.549420,d5e38725-610f-4f2c-8b91-6cc931a1e782,173bd1ca-0fef-4f39-871e-9caec7952a67
5,Criteria for homelessness detemination and doc...,Thank you for your thoughtful question. Based ...,None,The criteria for homelessness determination in...,1,0,1,4.661774,1f13a8f1-fc6e-4838-9834-d3db4a8fcd54,b2a96537-1bca-4e9c-a044-ae5632516b14
6,What are the dischare conditions that make a p...,Thank you for your thoughtful question. Based ...,None,"According to the context, a person is consider...",1,0,1,2.867942,30cb3f38-1d61-40ca-b8a3-46d7acf9d16c,7aeabab5-9c3d-4fcc-a480-33756faa9555
7,How does the FAFSA form determine veteran stat...,Thank you for your thoughtful question. Based ...,None,The FAFSA form asks students to declare their ...,1,1,0,10.202384,330c2806-e931-424a-8f7b-457c1134c2c5,61e2818d-e702-4580-a959-912366e2354b
8,How does the Department handle the disclosur o...,Thank you for your thoughtful question. Based ...,None,The disclosure of their FTI by the IRS to the ...,0,0,1,2.477212,4ac5b6f7-372d-44c7-a132-351d494d0929,a78f50e4-2ba3-4143-8f2a-2d172aa2167a
9,What role does the FAFSA play in the financial...,Thank you for your question! Based on the cont...,None,The FAFSA is used to produce output documents ...,1,1,1,2.996814,fb5d3d75-4022-4a25-a311-d99262f8ac2c,f87e0096-b932-43df-a500-f4f7d9063d63


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

✅✅✅
### Before Improvements (Default Chain)

image.png

#### After Improvements (Empathy RAG Chain)  

image.png

### The first chain is the more precise and utilitarian chain, but at the cost of warmth and slightly higher spend. The second chain is much more empethetic but it has slightly more errors that the first one. 

